In [11]:
def find_bigrams(input_list):
    return zip(input_list, input_list[1:])

def numpy_fillna(data):
    # Get lengths of each row of data
    lens = np.array([len(i) for i in data])

    # Mask of valid places in each row
    mask = np.arange(lens.max()) < lens[:,None]

    # Setup output array and put elements from data into masked positions
    out = np.zeros(mask.shape, dtype=data.dtype)
    out[mask] = np.concatenate(data)
    return out    

def tokenize(s):
    return tokens_re.findall(s)

    #args s = tweets, lowercase=False(keeps as in tweets) / True(changes all to lowercase)
def preprocess(s, lowercase=True):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

import tweepy
import csv
import string
import re
import pickle
import time
import numpy as np
import matplotlib.pyplot as plt
import pickle

from itertools import chain
from scipy import sparse
from IPython import display
from sklearn import metrics
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.cross_validation import cross_val_score, train_test_split

#Auth details from Twitter app http://twitclassify.com/
CONSUMER_KEY = "90bt0hlKzf3zWi5aPJ5ELZOGL"
CONSUMER_SECRET = "H4ycskJVRjcesIaZBVUFdTClsucdMEyPcZyQT7Xjvou3LWwBg2"
ACCESS_KEY = "303590178-hw3mgzNW08xAvbrVW4Zm8VP4uKpU9qyfG01ynkzi"
ACCESS_SECRET = "6wJw3hGpFAgR77nk6Hw4UjCGi96OtOZhnXjm1iMcptQ3i"

authid = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
authid.set_access_token(ACCESS_KEY, ACCESS_SECRET)
myApi = tweepy.API(authid, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)

punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'if', 'i', '’', '“', '”', 'it', 'the', '…', 'us', ':)', '—',
                                                   ':/', 'il', '', '≠', '•']

# Source taken from https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

data_train = np.genfromtxt("trainingfriends.tsv", delimiter='\t', dtype= str)

twitHandle = data_train[:,1] # this gets the tweet handles [column B]
y = data_train[:,0]

### Lists ###
desc_train_set = []
li = []
li_desc = []
uniqueWords = []
uniqueDict= {}
convList = []
bigList = []
userCap = []
userList = []
user_friend_id = []
friend_id = []
friend_id_list = []

userCap = userCap + [twitHandle[0]] # insert first element to list because 0 % 15 == 0 but should be treated as != 0

# take every 15 users, put into nested list eg. [[first 15 users][second 15 users]...[n 15 users]]
for i in range(1,len(twitHandle)+1):
    if(i == len(twitHandle)):
        userList.append(userCap) # append 15 users into an index of userList
        
    elif (i % 15 != 0):
        userCap = userCap + [twitHandle[i]]
        
    elif (i % 15 == 0):
        userList.append(userCap) # append 15 users into an index of userList
        userCap = [] # empty userCap (user capacity) list
        userCap = userCap + [twitHandle[i]] # add current iteration, i, element into userCap
print(userList)

for index in userList:
    for user_name in index:
        plan = myApi.get_user(screen_name=user_name) 

        for friend in plan.friends():
            friend_id = friend_id + [friend.id_str]
        
        user_friend_id.append(friend_id)
        friend_id = []
        time.sleep(60)

count = 0
for i in range(len(user_friend_id)):
    userProfile = myApi.lookup_users(user_ids = user_friend_id[i])
    for user in userProfile:
        count += 1
        #user profile description
        terms_stop = [term for term in preprocess(user.description) 
                      if term not in stop and
                      not term.startswith(('#', '@', 'http', 'https'))]

        #stores all terms into a list
        li_desc = li_desc + terms_stop
        friend_id = friend_id + terms_stop
        
        if (count == 20):
            # separates each descriptions to their respective users
            # add list of desc for each user into desc train set ie. desc_train_set = [[desc 1][desc 2]...[desc n]]
            print("I'm in!")
            desc_train_set.append(friend_id)
            friend_id = []
            count = 0

        
    # Append all unique unigram terms into uniqeWords list
    for j in range(len(li_desc)):
        for word in li_desc:
            if not word in uniqueWords:
                uniqueWords.append(word)
        
    # empty the list for next iteration    
    li_desc = []
    
print("Finished getting users!")
bigrams = list(chain.from_iterable(desc_train_set))
bigrams = list(find_bigrams(bigrams))

# Append all unique bigram terms into uniqeWords list
for i in range(len(bigrams)):
    print(i)
    for word in bigrams:
        if not word in uniqueWords:
            uniqueWords.append(word)

# Initializes a key, value of (eg. {term: id} ) into dictionary
for i,j in enumerate(uniqueWords):
    uniqueDict[i+1] = (j,i)
    
for j in range(len(uniqueDict)): # to create a single dictionary
    uniqueDict[j+1] = uniqueDict[j+1][0]

print("Finished creating dictionary!")
print(uniqueDict)
# Save to .pickle
with open('friendDict.pkl', 'wb') as f:
    pickle.dump(uniqueDict, f)

# Load from .pickle
with open('friendDict.pkl','rb') as f:
    uniqueDict = pickle.load(f)

inv_map = {v: k for k, v in uniqueDict.items()} # to reverse dict ordering
for index,word in enumerate(desc_train_set):         # iterate over set    
    for key, value in uniqueDict.items():
        if value in word:                      # see if word exists in dictionary
            convList.append(key)
    bigList.append(convList)
    convList = []
    
print("Done conversion!")

desc_train_set = np.asarray(bigList)
print(desc_train_set)

print(desc_train_set.shape)
desc_train_set = numpy_fillna(desc_train_set) # Match dimensions of array with highest no. of feature
print(desc_train_set.shape)

#splits data into training and test set
trainX, testX, train_y, test_y = train_test_split(desc_train_set, y, test_size=0.5)

[['21', '_cingraham', '_RoySD', 'billatnapier', '1954swilliamson', '2azizati', '_jdavenport', '_PaulHayward', '7OaksChronicle', 'AaronBesecker', 'acadvertiser', 'BelcherBlog', 'twitter', 'walmart', 'GM'], ['generalelectric', 'Target', 'WhirlpoolCorp', 'HackneyAbbott', 'Debbie_abrahams', 'DavidStewartMSP', 'heidi_mp', 'rushanaraali', 'grahamjones_mp', 'CallumSwanny', 'Callum_hird', 'TomCayman', 'CMaudsley', 'callumbaker', 'AnnabelCallum']]
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
I'm in!
Finished getting users!
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100

In [13]:
from sklearn.svm import LinearSVC
import operator
import itertools

# Source: https://stackoverflow.com/questions/1518522/python-most-common-element-in-a-list
# by user Alex Martelli
def most_common(L):
    # get an iterable of (item, iterable) pairs
    SL = sorted((x, i) for i, x in enumerate(L))
    # print 'SL:', SL
    groups = itertools.groupby(SL, key=operator.itemgetter(0))
    # auxiliary function to get "quality" for an item
    def _auxfun(g):
        item, iterable = g
        count = 0
        min_index = len(L)
        for _, where in iterable:
            count += 1
            min_index = min(min_index, where)
        # print 'item %r, count %r, minind %r' % (item, count, min_index)
        return count, -min_index
    # pick the highest-count/earliest item
    result = max(groups, key=_auxfun)[0]
    return result

# fit a SVM model to the data
modelSVM = LinearSVC(penalty='l2', loss='squared_hinge', dual=False, tol=0.0001, C=5.0, multi_class='ovr',
                     fit_intercept=True, intercept_scaling=1, class_weight=None, verbose=0,
                     random_state=None, max_iter=1000)
modelSVM.fit(trainX, train_y)
print(test_y)
print(train_y)
print(modelSVM)
# make predictions
expected = test_y
predicted = modelSVM.predict(testX)
# summarize the fit of the model
# Classification report
num = most_common(predicted)

if(num == '1'):
    community = "Academic"
elif(num == '2'):
    community = "Media"
elif(num == '3'):
    community = "Business"
elif(num == '4'):
    community = "Politic"
elif(num == '5'):
    community = "Citizen"

print("Community: ", community)
class_report = metrics.classification_report(expected, predicted)
print("Classification report \n", class_report)

# Confusion Metric
confMatrix = metrics.confusion_matrix(expected, predicted)
print("Confusion Metric = \n", confMatrix)

# Accuracy Score
print("Accuracy = ", metrics.accuracy_score(expected, predicted))

#True Positive Rate
tpr = (confMatrix[0,0] / (confMatrix[1,0] + confMatrix[0,0]))
print("Sensitivity = ", tpr)

#True Negative Rate
tnr = (confMatrix[1,1] / (confMatrix[0,1] + confMatrix[1,1]))
print("Specificity = ", tnr)

['5' '1' '2' '4' '2' '1' '2' '5' '2' '5' '4' '3' '1' '3' '1']
['3' '4' '2' '4' '5' '1' '5' '3' '1' '4' '2' '4' '5' '3' '3']
LinearSVC(C=5.0, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
Community:  Academic
Classification report 
              precision    recall  f1-score   support

          1       0.43      0.75      0.55         4
          2       0.33      0.25      0.29         4
          3       0.00      0.00      0.00         2
          4       0.00      0.00      0.00         2
          5       0.33      0.33      0.33         3

avg / total       0.27      0.33      0.29        15

Confusion Metric = 
 [[3 0 1 0 0]
 [3 1 0 0 0]
 [1 1 0 0 0]
 [0 0 0 0 2]
 [0 1 1 0 1]]
Accuracy =  0.333333333333
Sensitivity =  0.5
Specificity =  1.0


C:\Users\Hilmi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
from sklearn.naive_bayes import MultinomialNB

#Multinomial Naive Bayes Classifier full data

# fit a SVM model to the data
modelMNB = MultinomialNB()
modelMNB.fit(trainX, train_y)
print(modelMNB)
# make predictions
expected = test_y
predicted = modelMNB.predict(testX)
# summarize the fit of the model
# Classification report
print("Predicted: ", predicted)
print("Classification report ", metrics.classification_report(expected, predicted))

# Confusion Metric
confMatrix = metrics.confusion_matrix(expected, predicted)
print("Confusion Metric = \n", confMatrix)

# Accuracy Score
print("Accuracy = ", metrics.accuracy_score(expected, predicted))

#True Positive Rate
tpr = (confMatrix[0,0] / (confMatrix[1,0] + confMatrix[0,0]))
print("Sensitivity = ", tpr)

#True Negative Rate
tnr = (confMatrix[1,1] / (confMatrix[0,1] + confMatrix[1,1]))
print("Specificity = ", tnr)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Predicted:  ['1' '5' '5' '5' '2' '1' '5' '1' '5' '2' '5' '4' '2' '1' '4']
Classification report               precision    recall  f1-score   support

          1       0.25      0.25      0.25         4
          2       0.33      0.25      0.29         4
          3       0.00      0.00      0.00         2
          4       0.00      0.00      0.00         2
          5       0.00      0.00      0.00         3

avg / total       0.16      0.13      0.14        15

Confusion Metric = 
 [[1 1 0 1 1]
 [0 1 0 0 3]
 [1 0 0 1 0]
 [0 0 0 0 2]
 [2 1 0 0 0]]
Accuracy =  0.133333333333
Sensitivity =  1.0
Specificity =  0.5


C:\Users\Hilmi\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
from sklearn.linear_model import LogisticRegression

# fit a Logistic Regression model to the data
modelLR = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True,
                   intercept_scaling=1, class_weight=None, random_state=None,
                   solver='liblinear', max_iter=100, multi_class='ovr', verbose=0, warm_start=False, n_jobs=1)

modelLR.fit(trainX, train_y)
# make predictions
expected = test_y
predicted = modelLR.predict(testX)
# summarize the fit of the model
# Classification report
print("Predicted: ", predicted)
print("Classification report ", metrics.classification_report(expected, predicted))

# Confusion Metric
confMatrix = metrics.confusion_matrix(expected, predicted)
print("Confusion Metric = \n", confMatrix)

# Accuracy Score
print("Accuracy = ", metrics.accuracy_score(expected, predicted))

#True Positive Rate
tpr = (confMatrix[0,0] / (confMatrix[1,0] + confMatrix[0,0]))
print("Sensitivity = ", tpr)

#True Negative Rate
tnr = (confMatrix[1,1] / (confMatrix[0,1] + confMatrix[1,1]))
print("Specificity = ", tnr)

Predicted:  ['5' '3' '1' '5' '2' '1' '1' '3' '1' '2' '4' '2' '1' '1' '1']
Classification report               precision    recall  f1-score   support

          1       0.43      0.75      0.55         4
          2       0.33      0.25      0.29         4
          3       0.00      0.00      0.00         2
          4       1.00      0.50      0.67         2
          5       0.50      0.33      0.40         3

avg / total       0.44      0.40      0.39        15

Confusion Metric = 
 [[3 0 1 0 0]
 [3 1 0 0 0]
 [1 1 0 0 0]
 [0 0 0 1 1]
 [0 1 1 0 1]]
Accuracy =  0.4
Sensitivity =  0.5
Specificity =  1.0
